## Getting Started
First generating some test data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from annotation_driven_dataframe_calcs.annotation_driven_dataframe_calcs import expand_for_timesteps
from annotation_driven_dataframe_calcs.column_names import TIMESTEP_NO, ACCOUNT_NO, PARAM_C, PARAM_B, PARAM_A

fake_data = pd.DataFrame({
    ACCOUNT_NO: [1, 2, 3, 4],
    PARAM_A: [201908, 201907, 201906, 201905],
    PARAM_B: [10, 15, 20, 25],
    PARAM_C: [5, 6, 7, 8]
})

In [3]:
result = expand_for_timesteps(fake_data, 1, 3)
result

COLUMN_A  COLUMN_B  COLUMN_C
ACCT_NO TIMESTEP_NO                              
1       1              201908        10         5
        2              201908        10         5
        3              201908        10         5
2       1              201907        15         6
        2              201907        15         6
        3              201907        15         6
3       1              201906        20         7
        2              201906        20         7
        3              201906        20         7
4       1              201905        25         8
        2              201905        25         8
        3              201905        25         8

In [8]:
from annotation_driven_dataframe_calcs.first_stage_calcs import generate_series_a_prime

new_series = generate_series_a_prime(result)
new_series

ACCT_NO  TIMESTEP_NO
1        1              1.0
         2              1.0
         3              1.0
2        1              1.0
         2              1.0
         3              1.0
3        1              1.0
         2              1.0
         3              1.0
4        1              1.0
         2              1.0
         3              1.0
Name: COLUMN_A, dtype: float64